In [ ]:
# Step 1: Install elan (Lean toolchain manager) for Windows
import os
import subprocess
import urllib.request
import zipfile

# Download elan for Windows
elan_url = "https://github.com/leanprover/elan/releases/latest/download/elan-x86_64-pc-windows-msvc.exe"
elan_path = "elan-installer.exe"

print("Downloading elan installer...")
urllib.request.urlretrieve(elan_url, elan_path)

# Run the installer
print("Running elan installer...")
subprocess.run([elan_path, "-y"], check=True)

# Add elan to PATH for this session
elan_bin_path = os.path.expanduser("~/.elan/bin")
os.environ["PATH"] = elan_bin_path + ";" + os.environ["PATH"]

# Verify the installation
print("Verifying Lean installation...")
result = subprocess.run(["lean", "--version"], capture_output=True, text=True)
print(f"Lean version: {result.stdout}")

def setup_lean_project(project_dir="./lean_project"):
    """
    Creates a Lean project in the planner directory,
    configures it to use Mathlib, and downloads pre-compiled library files.
    """
    print(f"--- Setting up Lean project in: {project_dir} ---")
    os.makedirs(project_dir, exist_ok=True)
    os.makedirs(os.path.join(project_dir, "MinimalTheoremProject"), exist_ok=True)

    # Content for the lakefile.lean
    lakefile_content = """import Lake
open Lake DSL

package «minimal_theorem_project» where
  -- add package configuration options here

require mathlib from git
  "https://github.com/leanprover-community/mathlib4.git" @ "v4.10.0"

@[default_target]
lean_lib «MinimalTheoremProject» where
  -- add library configuration options here"""
  
    # Content for the theorem file
    theorem_content = """-- Minimal Lean 4 project for theorem planning testing
import Mathlib.Data.Real.Basic
import Mathlib.Tactic

-- Test theorem for the planner system  
theorem mathd_algebra_478 (b h v : ℝ) (h₀ : 0 < b ∧ 0 < h ∧ 0 < v) (h₁ : v = 1 / 3 * (b * h)) (h₂ : b = 30) (h₃ : h = 13 / 2) : v = 65 := by
  sorry"""

    # Write the lakefile
    with open(os.path.join(project_dir, "lakefile.lean"), "w") as f:
        f.write(lakefile_content)
        
    # Write the theorem file
    with open(os.path.join(project_dir, "MinimalTheoremProject", "TestTheorems.lean"), "w") as f:
        f.write(theorem_content)

    # Run `lake exe cache get` to download Mathlib's pre-compiled files
    print("--- Downloading Mathlib cache (this may take a few minutes)... ---")
    try:
        subprocess.run(
            ["lake", "exe", "cache", "get"],
            cwd=project_dir,
            check=True,
            capture_output=True,
            text=True
        )
        print("--- Mathlib cache downloaded successfully. ---")
        
        # Build the project
        print("--- Building project... ---")
        subprocess.run(
            ["lake", "build"],
            cwd=project_dir,
            check=True,
            capture_output=True,
            text=True
        )
        print("--- Project built successfully. ---")
        
    except subprocess.CalledProcessError as e:
        print("❌ Error setting up Lean project.")
        print(f"--- STDOUT ---\n{e.stdout}")
        print(f"--- STDERR ---\n{e.stderr}")
        raise  # Stop execution if setup fails

    return project_dir

# --- Call this function to set up the project ---
lean_project_path = setup_lean_project()
print(f"Lean project ready at: {lean_project_path}")

'sh' is not recognized as an internal or external command,
operable program or batch file.


Lean (version 4.24.0, x86_64-w64-windows-gnu, commit 797c613eb9b6d4ec95db23e3e00af9ac6657f24b, Release)
--- Setting up Lean project in: /tmp/lean_project ---
--- Downloading Mathlib cache (this may take a few minutes)... ---
❌ Error setting up Mathlib cache.
--- STDOUT ---

--- STDERR ---
error: Tried to read file 'C:\tmp\lean_project\lakefile.lean' containing non UTF-8 data.



CalledProcessError: Command '['lake', 'exe', 'cache', 'get']' returned non-zero exit status 1.